In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('clinicsales.csv')
data.describe()

clinicnumber   treatment    purchase  numdoctors  avgpanelsize  \
count    322.000000  322.000000  322.000000  322.000000    322.000000   
mean     161.500000    1.015528    0.276398    3.934783   2303.751553   
std       93.097619    0.822053    0.651862    1.170284    141.683691   
min        1.000000    0.000000    0.000000    1.000000   2050.000000   
25%       81.250000    0.000000    0.000000    3.000000   2186.000000   
50%      161.500000    1.000000    0.000000    4.000000   2308.000000   
75%      241.750000    2.000000    0.750000    5.000000   2419.750000   
max      322.000000    2.000000    9.000000    7.000000   2546.000000   

         distance  
count  322.000000  
mean    35.313665  
std     43.026230  
min      2.000000  
25%      9.000000  
50%     21.000000  
75%     45.750000  
max    437.000000

When we look at this, we can see that roughly 28% of the clinics purchased from us during this experiment (we do not yet know how this outcome metric varies by treatment). The average number of doctors at a clinic is just under 4 with a standard deviation of a bit above 1. The average panel size for doctors is around 2,300 with a standard deviation of 142. Average distance is 35, but with a big standard deviation of 43.

But hold on! Why is max(purchase) = 9, as purchase should be a dummy variable that only takes values of 0 (for not purchase) or 1 (for purchase).

This is an error that Dan deliberately inserted into this problem to see if you could spot it. It mimics what sometimes happens in real-world data where there is some simple coding error or in legacy (i.e., old) systems where people sometimes used numerical codes like 9 or 999 to denote missing data. As we go below we will look at how the results are affected if we don’t catch this data error.

In practice you would want to investigate an error like this and figure out if it should be a 0 or 1, and how sensitive your reuslts are to how you handle the error.

Here let’s handle the error by dropping that row from our data frame:

In [3]:
data = data[data['purchase']!=9]
data.describe()

clinicnumber   treatment    purchase  numdoctors  avgpanelsize  \
count    321.000000  321.000000  321.000000  321.000000    321.000000   
mean     161.732087    1.012461    0.249221    3.931464   2303.267913   
std       93.149627    0.821489    0.433237    1.170593    141.638437   
min        1.000000    0.000000    0.000000    1.000000   2050.000000   
25%       81.000000    0.000000    0.000000    3.000000   2186.000000   
50%      162.000000    1.000000    0.000000    4.000000   2308.000000   
75%      242.000000    2.000000    0.000000    5.000000   2419.000000   
max      322.000000    2.000000    1.000000    7.000000   2546.000000   

         distance  
count  321.000000  
mean    35.383178  
std     43.075293  
min      2.000000  
25%      9.000000  
50%     21.000000  
75%     46.000000  
max    437.000000

# 1. Evidence on the validity of the experiment / showing that the randomization was successful.

In [4]:
print( data.mean() )

balance_table = data[ ['treatment', 'numdoctors', 'avgpanelsize', 'distance'] ].groupby('treatment').mean()
balance_table['N'] = data.groupby('treatment')['clinicnumber'].count()
print(balance_table.T)

clinicnumber     161.732087
treatment          1.012461
purchase           0.249221
numdoctors         3.931464
avgpanelsize    2303.267913
distance          35.383178
dtype: float64
treatment               0            1            2
numdoctors       3.754717     3.980952     4.054545
avgpanelsize  2291.792453  2282.619048  2334.036364
distance        38.028302    35.161905    33.045455
N              106.000000   105.000000   110.000000


The means are quite similar across the treatment arms, and so are the sample sizes. The data appear balanced on observable characteristics, as we would expect if the randomizaiton were succesful.

# 2. An exhibit showing the means of all outcomes in each treatment arm, as well as 95% confidence intervals for the means.

In [5]:
p_means = data.groupby('treatment')['purchase'].agg(
    mean = ('mean'),
    std = ('std'),
    N  = ('size')

)
p_means['se'] = p_means['std']/np.sqrt(p_means['N'])
p_means['lb'] = p_means['mean'] - 1.96*p_means['se']
p_means['ub'] = p_means['mean'] + 1.96*p_means['se']

print(p_means[['mean', 'lb', 'ub']])

               mean        lb        ub
treatment                              
0          0.122642  0.059898  0.185385
1          0.333333  0.242732  0.423934
2          0.290909  0.205644  0.376174


# 3. An exhibit showing the ATEs for each outcome, as well las 95% confidence intervals for the ATE.

In [6]:
p_ate = p_means['mean'] - p_means['mean'][0]
p_ate = p_ate.to_frame(name='ate')
print(p_ate)

p_ate['se'] = np.sqrt( p_means['se']**2 + p_means['se'][0]**2 )
p_ate['lb'] = p_ate['ate'] - 1.96*p_ate['se']
p_ate['ub'] = p_ate['ate'] + 1.96*p_ate['se']

print(p_ate[['ate', 'lb', 'ub']][1:4])

                ate
treatment          
0          0.000000
1          0.210692
2          0.168268
                ate        lb        ub
treatment                              
1          0.210692  0.100486  0.320898
2          0.168268  0.062405  0.274130


# 4. A brief (1-3 sentence is fine) discussion of the business implications of the experiment. What should we do with the findings.

Based on these results our best estimate is that the “easier to use” pitch (Treatment 1) is the most effective approach. The results of this randomized controlled test indicate which pitch angle is most effective in converting clinics to purchase the product. With this information, EMS should consider standardizing their opening pitch based on the approach that yielded the highest conversion rate. Implementing the most successful pitch across the sales team could lead to improved sales efficiency and a higher rate of clinic partnerships. Additionally, understanding the effectiveness of each pitch approach provides insights that can be leveraged in training new sales representatives to maximize engagement and conversion during initial clinic visits.

# 5. A brief (1-3 sentence is fine) about possible issues. Is the outcome metric ideal, or what would be better? Did you spot anything troubling in the balance checks or results?

There are fairly wide confidence intervals around our estimates of these treatment effects that overlap substantially. So we do not want to be overconfident either about the level of increased sales we can expect when we start using this pitch going forward (best estimate is 21%, but not surprising to see something meaningfully higher or lower). We also do not want to be overconfident that we know that this pitch is better than the “fewer errors” pitch (Treatment 2). The data is consistent with the possibility that both pitches are equally effective.

There could be some other measures the EMS company want to consider to evaluate different aspects of the proposed treatments. For example, the total amount of purchase in dollars after the call, rather than simply indicator of purchase or not. We could also measure if the treatments results in a follow up meeting (for cases where there was not an immediate sale), the level of satisfaction of the call, and even the feedback or return rates after the purchase. We might further exam the sales over the next few weeks or months in case that there might be lag-over effect, or sales changes in sales of other products sold by ESA to test any spill-over effects.